In [4]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch.client import MlClient
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from urllib.request import urlopen
import json
from pathlib import Path
import os

# Connect to Elasticsearch

In [5]:
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")

In [6]:
url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"
client = Elasticsearch(url, ca_certs = "./http_ca.crt", verify_certs = True)
 
print(client.info())

{'name': 'liuxgm.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'n1BjmRPcR2GObT6ZMbJ9xA', 'version': {'number': '8.11.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'd9ec3fa628c7b0ba3d25692e277ba26814820b20', 'build_date': '2023-11-04T10:04:57.184859352Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Set the model name from Hugging Face and task type
# open ai detector model - developed by open ai https://github.com/openai/gpt-2-output-dataset/tree/master/detector

In [13]:
hf_model_id ='roberta-base-openai-detector'
tm = TransformerModel(model_id=hf_model_id, task_type="text_classification")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

#Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
STAGE:2023-12-23 18:11:58 87849:2048871 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-12-23 18:11:59 87849:2048871 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-12-23 18:11:59 87849:2048871 ActivityProfilerController.cpp:322] Completed Stage: Post Pr

  0%|          | 0/476 [00:00<?, ? parts/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/eland/ml/pytorch/_pytorch_model.py:105: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  self._client.ml.put_trained_model_definition_part(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/eland/ml/pytorch/_pytorch_model.py:83: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  self._client.perform_request(
/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_87849/924992814.py:17: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)


{'assignment': {'task_parameters': {'model_id': 'roberta-base-openai-detector',
   'deployment_id': 'roberta-base-openai-detector',
   'model_bytes': 498663888,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '498663888b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 498596904,
   'per_allocation_memory_bytes': 695458988},
  'routing_table': {'L9iKlXXzQgC_jV23VTtmjA': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2023-12-23T10:12:31.863175Z',
  'max_assigned_allocations': 1}}

# Set the model name from Hugging Face and task type
# sentence-transformers model

In [14]:
hf_model_id='sentence-transformers/all-mpnet-base-v2'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

# Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

STAGE:2023-12-23 18:19:10 87849:2048871 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-12-23 18:19:10 87849:2048871 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-12-23 18:19:10 87849:2048871 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


  0%|          | 0/416 [00:00<?, ? parts/s]

/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_87849/837366150.py:17: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)


{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__all-mpnet-base-v2',
   'deployment_id': 'sentence-transformers__all-mpnet-base-v2',
   'model_bytes': 435655636,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '435655636b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 435587600,
   'per_allocation_memory_bytes': 757850304},
  'routing_table': {'L9iKlXXzQgC_jV23VTtmjA': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2023-12-23T10:19:37.377013Z',
  'max_assigned_allocations': 1}}

# source index

In [16]:
client.indices.create(
index="plagiarism-docs",
mappings= {
    "properties": {
        "title": {
            "type": "text",
            "fields": {
                "keyword": {
                "type": "keyword"
                }
            }
        },
        "abstract": {
            "type": "text",
            "fields": {
                "keyword": {
                "type": "keyword"
                }
            }
        },
        "url": {
            "type": "keyword"
        },
        "venue": {
            "type": "keyword"
        },
         "year": {
            "type": "keyword"
        }
    }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'plagiarism-docs'})

# ingest pipeline

In [17]:
client.ingest.put_pipeline(
    id="plagiarism-checker-pipeline",
    processors = [
    {
      "inference": { #for ml models - to infer against the data that is being ingested in the pipeline
        "model_id": "roberta-base-openai-detector", #text classification model id
        "target_field": "openai-detector", # Target field for the inference results
        "field_map": { #Maps the document field names to the known field names of the model.
        "abstract": "text_field" # Field matching our configured trained model input. Typically for NLP models, the field name is text_field.
        }
      }
    },
    {
      "inference": {
        "model_id": "sentence-transformers__all-mpnet-base-v2", #text embedding model model id
        "target_field": "abstract_vector", # Target field for the inference results
        "field_map": { #Maps the document field names to the known field names of the model.
        "abstract": "text_field" # Field matching our configured trained model input. Typically for NLP models, the field name is text_field.
        }
      }
    }

  ]
)

ObjectApiResponse({'acknowledged': True})

# Create plagiarism checker index

In [18]:
client.indices.create(
index="plagiarism-checker",
mappings={
"properties": {
    "title": {
        "type": "text",
        "fields": {
            "keyword": {
                "type": "keyword"
            }
        }
    },
    "abstract": {
        "type": "text",
        "fields": {
            "keyword": {
                "type": "keyword"
            }
        }
    },
    "url": {
        "type": "keyword"
    },
    "venue": {
        "type": "keyword"
    },
    "year": {
        "type": "keyword"
    },
    "abstract_vector.predicted_value": { # Inference results field, target_field.predicted_value
    "type": "dense_vector",
    "dims": 768, # embedding_size
    "index": "true",
    "similarity": "dot_product" #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
         }
  }
}
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'plagiarism-checker'})

# Write source documents

In [19]:
# Load data into a JSON object
with open('emnlp2016-2018.json') as f:
   data_json = json.load(f)
 
print(f"Successfully loaded {len(data_json)} documents")

def create_index_body(doc):
    """ Generate the body for an Elasticsearch document. """
    return {
        "_index": "plagiarism-docs",
        "_source": doc,
    }

# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `plagiarism-docs` source index")

Successfully loaded 974 documents
Done indexing documents into `plagiarism-docs` source index


# Reindex with ingest pipeline

In [23]:
client.reindex(wait_for_completion=False,
               source={
                  "index": "plagiarism-docs"
    },
               dest= {
                  "index": "plagiarism-checker",
                  "pipeline": "plagiarism-checker-pipeline"
    }
)

ObjectApiResponse({'task': 'L9iKlXXzQgC_jV23VTtmjA:1053110'})

# duplicated text - direct plagiarism

In [24]:
model_text = 'Understanding and reasoning about cooking recipes is a fruitful research direction towards enabling machines to interpret procedural text. In this work, we introduce RecipeQA, a dataset for multimodal comprehension of cooking recipes. It comprises of approximately 20K instructional recipes with multiple modalities such as titles, descriptions and aligned set of images. With over 36K automatically generated question-answer pairs, we design a set of comprehension and reasoning tasks that require joint understanding of images and text, capturing the temporal flow of events and making sense of procedural knowledge. Our preliminary results indicate that RecipeQA will serve as a challenging test bed and an ideal benchmark for evaluating machine comprehension systems. The data and leaderboard are available at http://hucvl.github.io/recipeqa.'

response = client.search(index='plagiarism-checker', size=1,
    knn={
        "field": "abstract_vector.predicted_value",
        "k": 9,
        "num_candidates": 974,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-mpnet-base-v2",
                "model_text": model_text
            }
        }
    }
)

for hit in response['hits']['hits']:
    score = hit['_score']
    title = hit['_source']['title']
    abstract = hit['_source']['abstract']
    openai = hit['_source']['openai-detector']['predicted_value']
    url = hit['_source']['url']

    if score > 0.9:
        print(f"\nHigh similarity detected! This might be plagiarism.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    elif score < 0.7:
        print(f"\nLow similarity detected. This might not be plagiarism.")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    else:
        print(f"\nModerate similarity detected.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

ml_client = MlClient(client)

model_id = 'roberta-base-openai-detector' #open ai text classification model

document = [
    {
        "text_field": model_text
    }
]

ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)

predicted_value = ml_response['inference_results'][0]['predicted_value']

if predicted_value == 'Fake':
    print("Note: The text query you entered may have been generated by AI.\n")


Moderate similarity detected.

Most similar document: 'Natural Language Comprehension with the EpiReader'

Abstract: We present the EpiReader, a novel model for machine comprehension of text. Machine comprehension of unstructured, real-world text is a major research goal for natural language processing. Current tests of machine comprehension pose questions whose answers can be inferred from some supporting text, and evaluate a model's response to the questions. The EpiReader is an end-to-end neural model comprising two components: the first component proposes a small set of candidate answers after comparing a question to its supporting text, and the second component formulates hypotheses using the proposed candidates and the question, then reranks the hypotheses based on their estimated concordance with the supporting text. We present experiments demonstrating that the EpiReader sets a new state-of-the-art on the CNN and Children's Book Test machine comprehension benchmarks, outperfor

/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_87849/283995233.py:54: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)


# similar text - paraphrase plagiarism

In [25]:
model_text = 'Comprehending and deducing information from culinary instructions represents a promising avenue for research aimed at empowering artificial intelligence to decipher step-by-step text. In this study, we present CuisineInquiry, a database for the multifaceted understanding of cooking guidelines. It encompasses a substantial number of informative recipes featuring various elements such as headings, explanations, and a matched assortment of visuals. Utilizing an extensive set of automatically crafted question-answer pairings, we formulate a series of tasks focusing on understanding and logic that necessitate a combined interpretation of visuals and written content. This involves capturing the sequential progression of events and extracting meaning from procedural expertise. Our initial findings suggest that CuisineInquiry is poised to function as a demanding experimental platform.'

response = client.search(index='plagiarism-checker', size=1,
    knn={
        "field": "abstract_vector.predicted_value",
        "k": 9,
        "num_candidates": 974,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-mpnet-base-v2",
                "model_text": model_text
            }
        }
    }
)

for hit in response['hits']['hits']:
    score = hit['_score']
    title = hit['_source']['title']
    abstract = hit['_source']['abstract']
    openai = hit['_source']['openai-detector']['predicted_value']
    url = hit['_source']['url']

    if score > 0.9:
        print(f"\nHigh similarity detected! This might be plagiarism.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    elif score < 0.7:
        print(f"\nLow similarity detected. This might not be plagiarism.")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    else:
        print(f"\nModerate similarity detected.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

ml_client = MlClient(client)

model_id = 'roberta-base-openai-detector' #open ai text classification model

document = [
    {
        "text_field": model_text
    }
]

ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)

predicted_value = ml_response['inference_results'][0]['predicted_value']

if predicted_value == 'Fake':
    print("Note: The text query you entered may have been generated by AI.\n")


Moderate similarity detected.

Most similar document: 'Sort Story: Sorting Jumbled Images and Captions into Stories'

Abstract: Temporal common sense has applications in AI tasks such as QA, multi-document summarization, and human-AI communication. We propose the task of sequencing -- given a jumbled set of aligned image-caption pairs that belong to a story, the task is to sort them such that the output sequence forms a coherent story. We present multiple approaches, via unary (position) and pairwise (order) predictions, and their ensemble-based combinations, achieving strong results on this task. We use both text-based and image-based features, which depict complementary improvements. Using qualitative examples, we demonstrate that our models have learnt interesting aspects of temporal common sense.

url: http://aclweb.org/anthology/D16-1091

Score:0.7819495

Note: The text query you entered may have been generated by AI.



/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_87849/4098368190.py:54: ElasticsearchWarning: Your license will expire in [5] days. Contact your administrator or update your license for continued use of features
  ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)
